In [1]:
from imports import cv2, os, np, tf, ImageDataGenerator, plt, pd, requests
import confidential

firebase_url = confidential.get_url()

response = requests.get(firebase_url)

data = response.json()
data_list = [value for key, value in data.items()]

df = pd.DataFrame(data_list)
df = df[["name", "url"]]

def download_image(url, name):
    response = requests.get(url)
    image_data = response.content
    array = bytearray(image_data)

    image = cv2.imdecode(np.asarray(array, dtype=np.uint8), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (180, 180))
    return image

images = []

for index, row in df.iterrows():
    image = download_image(row["url"], row["name"])
    images.append(image)

images = np.array(images)

2024-03-28 00:11:33.550280: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 00:11:33.550325: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 00:11:33.551209: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-28 00:11:33.557817: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 00:11:34.289184: W tensorflow/compiler/tf2

In [2]:
print(images.shape)

(2, 180, 180, 3)


In [3]:
precision = 50

def augment_images(images):
    train_datagen = ImageDataGenerator(rescale=1./255,
                                        rotation_range=40,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest')
    augmented_images = []
    for image in images:
        image = np.expand_dims(image, 0)
        i = 0
        for batch in train_datagen.flow(image, batch_size=1):
            augmented_images.append(batch)
            i += 1
            if i >= precision:
                break
    return augmented_images

augmented_images = augment_images(images)
augmented_images = np.array(augmented_images).reshape(-1, 180, 180, 3)

In [4]:
labels = df["name"].values
labels = np.repeat(labels, precision)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#encode labels with sklearn
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

model.fit(augmented_images, labels, epochs=50)

2024-03-28 00:11:36.503203: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


Epoch 1/50


2024-03-28 00:11:36.521857: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-28 00:11:36.521922: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-28 00:11:36.523610: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-28 00:11:36.523781: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-28 00:11:36.523831: I external/local_xla/xla/stream_executor

4/4 [==============================] - 3s 179ms/step - loss: 3.7468 - accuracy: 0.5800
Epoch 2/50
4/4 [==============================] - 0s 19ms/step - loss: 0.9169 - accuracy: 0.3800
Epoch 3/50
4/4 [==============================] - 0s 19ms/step - loss: 0.3992 - accuracy: 0.8600
Epoch 4/50
4/4 [==============================] - 0s 19ms/step - loss: 0.1593 - accuracy: 0.9400
Epoch 5/50
4/4 [==============================] - 0s 19ms/step - loss: 0.0221 - accuracy: 1.0000
Epoch 6/50
4/4 [==============================] - 0s 19ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 7/50
4/4 [==============================] - 0s 19ms/step - loss: 3.0522e-04 - accuracy: 1.0000
Epoch 8/50
4/4 [==============================] - 0s 19ms/step - loss: 2.0415e-05 - accuracy: 1.0000
Epoch 9/50
4/4 [==============================] - 0s 19ms/step - loss: 8.6936e-06 - accuracy: 1.0000
Epoch 10/50
4/4 [==============================] - 0s 19ms/step - loss: 2.7745e-05 - accuracy: 1.0000
Epoch 11/50
4/4 [======

In [8]:
#predict on given image
test_image = cv2.imread("test.jpg")
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
test_image = cv2.resize(test_image, (180, 180))
test_image = np.expand_dims(test_image, 0)

prediction = model.predict(test_image)
prediction = le.inverse_transform([np.argmax(prediction)])
print(prediction)

1/1 [==============================] - 0s 13ms/step
['Lennart']
